In [1]:
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests


# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.0


/Users/jinjunjie/anaconda/envs/IntroToTensorFlow/lib/python3.5/site-packages/ipykernel/__main__.py:15: UserWarning: No GPU found. Please use a GPU to train your neural network.


In [2]:
data_dir = './data'
helper.maybe_download_pretrained_vgg(data_dir)

In [3]:
def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'

    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)

    image_input = tf.get_default_graph().get_tensor_by_name(vgg_input_tensor_name)
    keep_prob = tf.get_default_graph().get_tensor_by_name(vgg_keep_prob_tensor_name)
    layer3_out = tf.get_default_graph().get_tensor_by_name(vgg_layer3_out_tensor_name)
    layer4_out = tf.get_default_graph().get_tensor_by_name(vgg_layer4_out_tensor_name)
    layer7_out = tf.get_default_graph().get_tensor_by_name(vgg_layer7_out_tensor_name)
    
    return (image_input, keep_prob, layer3_out, layer4_out, layer7_out)

In [4]:
tests.test_load_vgg(load_vgg, tf)

Tests Passed


In [9]:
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    vgg_layer7 = tf.layers.conv2d(vgg_layer7_out, num_classes, 1, padding='SAME',
                                  kernel_initializer=tf.truncated_normal_initializer(stddev = 0.01))

    # similar for vgg_layer4 and vgg_layer3 taking the corresponding vgg_layer4_out and vgg_layer3_out

    fcn_layer1 = tf.layers.conv2d_transpose(vgg_layer7, num_classes, 4, 2, 'SAME',
                                              kernel_initializer=tf.truncated_normal_initializer(stddev = 0.01))

    combined_layer1 = tf.add(vgg_layer4_out, fcn_layer1)

    fcn_layer2 = tf.layers.conv2d_transpose(combined_layer1, num_classes, 4, 2, 'SAME',
                                                kernel_initializer=tf.truncated_normal_initializer(stddev = 0.01))

    combined_layer2 = tf.add(vgg_layer3_out, fcn_layer2)

    output = tf.layers.conv2d_transpose(combined_layer2, num_classes, 16, 8, 'SAME',
                                              kernel_initializer=tf.truncated_normal_initializer(stddev = 0.01))
    return output

In [10]:
tests.test_layers(layers)

ValueError: Dimensions must be equal, but are 512 and 2 for 'Add' (op: 'Add') with input shapes: [?,?,?,512], [?,?,?,2].

In [ ]:
def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    
    logits = tf.reshape(nn_last_layer, (-1, num_classes))
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=correct_label))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cross_entropy_loss)
    return logits, optimizer, cross_entropy_loss

In [ ]:
tests.test_optimize(optimize)

In [ ]:
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    sess.run(tf.global_variables_initializer())
    for e in range(epochs):
        for x, y in get_batches_fn(batch_size):
            feed = {input_image: x,
                    correct_label: y,
                    learning_rate:0.001,
                    keep_prob: 0.6}
            loss, _ = sess.run([cross_entropy_loss, train_op], feed_dict=feed)
            print("Epoch: {}/{}".format(e+1, epochs),
                  "Training loss: {:.5f}".format(loss))


In [ ]:
tests.test_train_nn(train_nn)

In [ ]:
def run():
    num_classes = 2
    image_shape = (160, 576)
    epochs = 10
    batch_size = 3
    data_dir = './data'
    runs_dir = './runs'
    tests.test_for_kitti_dataset(data_dir)

    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)

    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/

    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    correct_label = tf.placeholder(tf.float32, name='correct_label')
    
    with tf.Session() as sess:
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

        # OPTIONAL: Augment Images for better results
        #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network

        # TODO: Build NN using load_vgg, layers, and optimize function
        image_input, keep_prob, layer3_out, layer4_out, layer7_out = load_vgg(sess, vgg_path)
        
        nn_last_layer = layers(layer3_out, layer4_out, layer7_out, num_classes)
        
        logits, optimizer, cross_entropy_loss = optimize(nn_last_layer, correct_label, learning_rate, num_classes)
        # TODO: Train NN using the train_nn function
        train_nn(sess, epochs, batch_size, get_batches_fn, optimizer, cross_entropy_loss, image_input,
                 correct_label, keep_prob, learning_rate)
        
        # TODO: Save inference data using helper.save_inference_samples
        helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, image_input)

        # OPTIONAL: Apply the trained model to a video

In [ ]:
run()